In [6]:
%load_ext autoreload
%autoreload 2


import torch

import torch.nn as nn
from torch.distributions import Categorical, Poisson, MixtureSameFamily
from matplotlib import pyplot as plt


# Cd to code
import os
import sys
os.chdir('/cluster/home/kheuto01/code/prob_diff_topk')
sys.path.append('/cluster/home/kheuto01/code/prob_diff_topk')

from datasets import example_datasets, to_numpy
from torch_perturb.torch_pert_topk import PerturbedTopK
from torch_models import MixtureOfPoissonsModel, MixtureOfTruncNormModel, torch_bpr_uncurried, deterministic_bpr

2024-07-15 15:15:34.322462: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-15 15:15:34.384247: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-15 15:15:34.384311: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-15 15:15:34.386171: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-15 15:15:34.399345: I tensorflow/core/platform/cpu_feature_guar

In [7]:
seed=360
# tracts/distributions
S=12
# history/features
H = 3
# total timepoints
T= 500
num_components=4
K=4

In [8]:
train_dataset, val_dataset, test_dataset = example_datasets(H, T, seed=seed)
train_X_THS, train_y_TS = to_numpy(train_dataset)

2024-07-15 15:16:02.887326: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-15 15:16:02.887369: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: s1cmp008.pax.tufts.edu
2024-07-15 15:16:02.887374: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: s1cmp008.pax.tufts.edu
2024-07-15 15:16:02.887415: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.129.3
2024-07-15 15:16:02.887442: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.129.3
2024-07-15 15:16:02.887448: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.129.3


In [9]:
# take inverse softplus
ideal_means = torch.tensor([0+1e-8, 7, 10, 100])
ideal_softinv_means = ideal_means + torch.log(-torch.expm1(-ideal_means))
ideal_scales = torch.tensor([0.2, 0.2, 0.2, 0.2])
ideal_softinv_scales = ideal_scales - 0.2 + torch.log(-torch.expm1(-ideal_scales + 0.2)) 
ideal_mix_weights = torch.log(1e-13 + torch.tensor(
                                [[0,1,0,0],
                                 [0,1,0,0],
                                 [0,1,0,0],
                                 [0,1,0,0],
                                 [0.3,0,0.7,0],
                                 [0.3,0,0.7,0],
                                 [0.3,0,0.7,0],
                                 [0.3,0,0.7,0],
                                 [0.9,0,0,0.1],
                                 [0.9,0,0,0.1],
                                 [0.9,0,0,0.1],
                                 [0.9,0,0,0.1]]))

In [10]:
model = MixtureOfTruncNormModel()
step_size = 0.1
optimizer = torch.optim.Adam(model.parameters(), lr=step_size)

#model.update_params(torch.cat([ideal_softinv_means, ideal_softinv_scales, ideal_mix_weights.view(-1)]))

In [11]:
M_score_func =  200
M_action = 200
train_T = train_y_TS.shape[0]
perturbed_top_K_func = PerturbedTopK(k=4)


In [12]:
losses = []
bprs = []
nlls = []

In [13]:

for epoch in range(2):
    mix_model = model()
    
    y_sample_TMS = mix_model.sample((train_T, M_score_func))
    y_sample_action_TMS = mix_model.sample((train_T, M_action))

    ratio_rating_TMS = y_sample_action_TMS/y_sample_action_TMS.sum(dim=-1, keepdim=True)
    ratio_rating_TS =  ratio_rating_TMS.mean(dim=1)
    ratio_rating_TS.requires_grad_(True)

    #pred_y_TS = torch.mean(y_sample_action_TMS, dim=1)
    #pred_y_TS.requires_grad_(True)

    def get_log_probs_baked(param):
        distribution = model.build_from_single_tensor(param)
        log_probs_TMS = distribution.log_prob(y_sample_TMS)

        return log_probs_TMS

    jac_TMSP = torch.autograd.functional.jacobian(get_log_probs_baked, (model.params_to_single_tensor()), strategy='forward-mode', vectorize=True)

    score_func_estimator_TMSP = jac_TMSP * ratio_rating_TMS.unsqueeze(-1)
    score_func_estimator_TSP = score_func_estimator_TMSP.mean(dim=1)    

    # get gradient of negative bpr_t  with respect to ratio rating_TS
    positive_bpr_T = torch_bpr_uncurried(ratio_rating_TS, torch.tensor(train_y_TS), K=4, perturbed_top_K_func=perturbed_top_K_func)
    negative_bpr = torch.mean(-positive_bpr_T)
    
    nll = torch.sum(-mix_model.log_prob( torch.tensor(train_y_TS)))

    
    loss = 500*negative_bpr + nll
    
    
    loss.backward()

    loss_grad_TS = ratio_rating_TS.grad

    gradient_TSP = score_func_estimator_TSP * torch.unsqueeze(loss_grad_TS, -1)
    gradient_P = torch.sum(gradient_TSP, dim=[0,1])

    gradient_tuple = model.single_tensor_to_params(gradient_P)

    for param, gradient in zip(model.parameters(), gradient_tuple):
        param.grad = gradient
    optimizer.step()

    deterministic_bpr_T = deterministic_bpr(ratio_rating_TS, torch.tensor(train_y_TS), K=K)
    det_bpr =torch.mean(deterministic_bpr_T)
    print(f'Epoch: {epoch}')
    print(f'det bpr: {det_bpr}')
    print(f'nll: {nll}')
    print(f'Loss: {loss}')
    losses.append(loss)
    bprs.append(det_bpr)
    nlls.append(nll)
        
    #model.update_params(model.params_to_single_tensor() - step_size * gradient_P)


    
    


Epoch: 0
det bpr: 0.47491520643234253
nll: 321912.4375
Loss: 321673.71875
Epoch: 1
det bpr: 0.48146504163742065
nll: 291136.21875
Loss: 290898.21875


In [4]:
torch.load('/cluster/home/kheuto01/code/prob_diff_topk/scratch/losses.pth')

[tensor(289367.4688, requires_grad=True),
 tensor(292217.5312, requires_grad=True),
 tensor(294833.4688, requires_grad=True),
 tensor(296400.2188, requires_grad=True),
 tensor(298546.8438, requires_grad=True),
 tensor(300956.8750, requires_grad=True),
 tensor(303040.0312, requires_grad=True),
 tensor(305312.0312, requires_grad=True),
 tensor(306492.3750, requires_grad=True),
 tensor(306622.1875, requires_grad=True)]

In [2]:
import torch


Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)



/cluster/tufts/hugheslab/kheuto01/mambaforge/envs/pip_k3/lib/python3.11/site-packages/IPython/core/completerlib.py:149: UserWarning: using rootmodules_cache requires you to install the `pickleshare` library.
  ip.db['rootmodules_cache'] = rootmodules_cache


In [16]:
positive_bpr_T
threshold = 0.55
#create flag where bpr > threshold
flag = positive_bpr_T > threshold

In [18]:
flag*positive_bpr_T

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.5918, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.5782, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.5648, 0.5814, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.5620, 0.0000, 0.0000, 0.0000, 0.5820, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5854,
        0.5522, 0.0000, 0.0000, 0.5573, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5634, 0.0000, 0.0000,
        0.0000, 0.5700, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.5807, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5802,
        0.5759, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [22]:
torch.nn.functional.softplus(torch.tensor([10.]))

tensor([10.0000])

In [ ]:
torch.rand()